## NLP
- 자연어 -> 컴퓨터 언어 (NLU)
  - Text Classification
  - POS Tagging
  - Sentiment Analysis
  - Machine Reading Comprehension
  - Named Entity Recognition
  - Semantic Parsing
- 컴퓨터 언어 -> 자연어 (NLG)
  - Language Modeling
  - Article Generation
- intersection
  - Chatbot
  - Summarization
  - Question Answering
  - Machine Translation

## Preprocessing
- 정제 -> Tokenization -> Subword Segmentation(word2index) -> Batchify -> Prediction -> Detokenization(index2word)  

### 데이터 수집
- Purchase
- Crwaling

## Regular Expression

In [4]:
# import sys
# import re

# def read_regex(fn):
#     regexs = []

#     f = open(fn, 'r')

#     for line in f:
#         if not line.startswith("#"):
#             tokens = line.split('\t')

#             if len(tokens) == 1:
#                 tokens += [' ']

#             tokens[0] = tokens[0][:-1] if tokens[0].endswith('\n') else tokens[0]
#             tokens[1] = tokens[1][:-1] if tokens[1].endswith('\n') else tokens[1]

#             regexs += [(tokens[0], tokens[1])]

#     f.close()

#     return regexs

# if __name__ == "__main__":
#     fn = sys.argv[1]
#     target_index = int(sys.argv[2])

#     regexs = read_regex(fn)

#     for line in sys.stdin:
#         if line.strip() != "":
#             columns = line.strip().split('\t')

#             for r in regexs:
#                 columns[target_index] = re.sub(r'%s' % r[0], r[1], columns[target_index].strip())

#             sys.stdout.write('\t'.join(columns) + "\n")
#         else:
#             sys.stdout.write('\n')

## Tokenizing
- token 길이가 짧을경우
  - Vocab 크기 감소, 희소성 문제 감소
  - OoV(Out of Vocabulary, test 과정에서 train에서 못 본 단어, \<UNK\>로 치환)가 줄어듬
  - Sequence의 길이가 김 -> 모델의 부담 증가
- 길 수록
  - 희소성 문제 증대
  - OoV 상승 및 Sequence의 길이가 짧아 모델의 부담 감소

- 정보량에 따라
  - 빈도가 높을 경우 하나의 token으로 표현
  - 빈도가 적을경우 잘게 쪼개어, 빈도가 높은 token으로 구성


## Subword Segmentation
- subword: 단어보다 작은 의미 단위
- 이를 위해 별도의 subword 사전이 필요

### Byte Pair Encoding(BPE) 알고리즘
- training corpus를 활용하여 bpe 모델 학습 후, 코퍼스에 적용
- pros
  - 희소성을 통계에 기반하여 효과적으로 낮춤
  - 언어별 특성에 대한 정보 없이 의미단위로 분절 가능
  - OoV를 없앨 수 있다.
 - cons
   - 학습데이터별로 BPE 모델도 생성됨  
- Training
  - 단어 사전 생성 (빈도 포함)
  - Character 단위로 분절 후, pair 별 빈도 카운트
  - 최빈도 pair를 골라 merge 수행
  - pair별 빈도 카운트 업데이트
  - 3 -> 반복  
- 한국어의 경우 띄어쓰기가 제멋대로라, tokenizing 후 진행하는 것이 좋음

## Align Pararrel Corpus
- 뉴스 기사, 영화 자막 등에서 대부분 문서 단위 matching 되어있고, 문장 단위는 X
- ex. 영문 뉴스 기사 <-> 한국 뉴스 기사

### Champollion
- 단어 변역 사전에 기반하여, 사전을 최대한 만족하는 align을 찾음
- 단어 번역 사전 -> Faebook MUSE
  - 서로 다른 언어의 두 문서를 비교하여, Embedding을 비교하여 단어 번역

## Mini-Batch
- (batch_size, length, |V(one-hot vector)|) -> one-hot vector의 index값만 가져와도 됌
- => (batch_size, length)
- 생성과정에서 sequence length에 맞춰 정렬하는 이유
  - mini-batch 내의 sqeunce 길이 차이가 많이 날 경우, \<pad\>만 채워진 문장 등으로 비효율
  - mini-batch를 shuffling 하여 random하게 학습
### 구성
- corpus를 길이에 따라 정렬
- token을 사전을 활용하여 str-index mapping
- mini-batch 크기대로 chunk
- 각 미니배치 별 텐서 구성 및 padding
- 학습시 shuffling하여 iterative하게 반환

### TorchText
- text 로딩용 라이브러리

In [10]:
from torchtext.legacy import data

class DataLoader(object):

    def __init__(
        self, train_fn,
        batch_size=64,
        valid_ratio=.2,
        device=-1,
        max_vocab=999999,
        min_freq=1,
        use_eos=False,
        shuffle=True
    ):
        super().__init__()

        self.label = data.Field(
            sequential=False,
            use_vocab=True,
            unk_token=None
        )
        
        self.text = data.Field(
            use_vocab=True,
            batch_first=True,
            include_lengths=False,
            eos_token='<EOS>' if use_eos else None
        )

        train, valid = data.TabularDataset(
            path=train_fn,
            format='tsv',
            fields=[
                ('label', self.label),
                ('text', self.text)
            ],
        ).split(split_ratio=(1 - valid_ratio))

        self.train_loader, self.valid_loader = data.BucketIterator.splits(
            (train, valid),
            batch_size=batch_size,
            device='cuda:%d' % device if device >= 0 else 'cpu',
            shuffle=shuffle,
            sort_key=lambda x: len(x.text),
            sort_within_batch=True
        )

        self.label.build_vocab(train)
        self.text.build_vocab(train, max_size=max_vocab, min_freq=min_freq)

In [14]:
loaders = DataLoader(
    train_fn='./review.sorted.uniq.refined.tok..shuf.train.tsv',
    batch_size=256,
    valid_ratio=.2,
    device=-1,
    max_vocab=999999,
    min_freq=5,
)

In [15]:
print("|train|=%d" % len(loaders.train_loader.dataset))
print("|valid|=%d" % len(loaders.valid_loader.dataset))

|train|=198023
|valid|=49506


In [16]:
print("|vocab|=%d" % len(loaders.text.vocab))
print("|label|=%d" % len(loaders.label.vocab))

|vocab|=17078
|label|=2


In [17]:
data = next(iter(loaders.train_loader))

print(data.text.shape)
print(data.label.shape)

torch.Size([256, 26])
torch.Size([256])


In [18]:
dir(loaders.text.vocab)

['UNK',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_default_unk_index',
 'extend',
 'freqs',
 'itos',
 'load_vectors',
 'lookup_indices',
 'set_vectors',
 'stoi',
 'unk_index',
 'vectors']

In [19]:
# stoi = string to index
loaders.text.vocab.stoi['배송']

18

In [20]:
# itos = index to string
loaders.text.vocab.itos[18]

'배송'

In [21]:
for i in range(50):
    word = loaders.text.vocab.itos[i]
    print('%5d: %s\t%d' % (i, word, loaders.text.vocab.freqs[word]))

    0: <unk>	0
    1: <pad>	0
    2: .	239871
    3: 고	137806
    4: 이	132059
    5: 하	113380
    6: 도	93596
    7: 네요	90800
    8: 좋	87899
    9: 에	84665
   10: 는	79185
   11: 가	64409
   12: 은	60747
   13: 는데	52134
   14: 아요	49545
   15: 게	49397
   16: 잘	48316
   17: 어요	45911
   18: 배송	43585
   19: 있	41532
   20: 했	38792
   21: 습니다	38620
   22: 안	35249
   23: 을	34890
   24: 한	32937
   25: ~	30584
   26: 구매	28857
   27: 같	27811
   28: 너무	27328
   29: 거	26898
   30: 합니다	26865
   31: 지	26450
   32: ..	24535
   33: 어	23914
   34: !	23645
   35: ,	23612
   36: 다	23467
   37: 가격	22765
   38: 되	22271
   39: ?	21930
   40: 것	21315
   41: 들	20855
   42: 제품	20774
   43: 았	20634
   44: 으로	20523
   45: 쓰	20198
   46: 아	20051
   47: 만	19993
   48: 로	19662
   49: 받	19629


In [22]:
print(data.text[-1])

tensor([1224,   20,   13,  390,  325, 6806,   54,   90,  419,   10,    0,  588,
         143,  991,  314,  191, 1473,  150,  148,   40,   27,  105,  808,  718,
         324,    1])


In [23]:
x = data.text[-1]
line = []
for x_i in x:
    line += [loaders.text.vocab.itos[x_i]]
    
print(' '.join(line))

혹시나 했 는데 별루 임 싸우 나 에서 나오 는 <unk> 로션 정도 50 대 ㅋㅋㅋ 아저씨 용 인 것 같 음 향도 영 ~~~ <pad>
